In [1]:
# define and move to dataset directory
# I changed the dir to match the repo. Note that I am using Windows, so *NIX user should change the following line.
datasetdir = '.\\dataset\\Plant_leaf_diseases_dataset_with_augmentation'
#import os
#os.chdir(datasetdir)

# import the needed packages
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import tensorflow as tf
from tensorflow import keras

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
import warnings

# check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /device:GPU:0


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 30

def generators(shape, preprocessing): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        horizontal_flip = True, 
        validation_split = 0.1)

    height, width = shape

    train_dataset = imgdatagen.flow_from_directory(
        datasetdir,
        target_size = (height, width),
        batch_size = batch_size,
        subset = 'training')

    val_dataset = imgdatagen.flow_from_directory(
        datasetdir,
        target_size = (height, width),
        batch_size = batch_size,
        subset = 'validation')
    
    return train_dataset, val_dataset

In [4]:
def plot_history(history, yrange):
    '''Plot loss and accuracy as a function of the epoch,
    for the training and validation datasets.
    '''
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Get number of epochs
    epochs = range(len(acc))

    # Plot training and validation accuracy per epoch
    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.title('Training and validation accuracy')
    plt.ylim(yrange)
    
    # Plot training and validation loss per epoch
    plt.figure()

    plt.plot(epochs, loss)
    plt.plot(epochs, val_loss)
    plt.title('Training and validation loss')
    
    plt.show()

__VGG19__

In [5]:
vgg19 = keras.applications.vgg19
conv_model = vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in conv_model.layers: 
    layer.trainable = False
x = keras.layers.Flatten()(conv_model.output)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dense(64, activation='relu')(x)
predictions = keras.layers.Dense(39, activation='softmax')(x)
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
full_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
train_dataset, val_dataset = generators((224,224), preprocessing=vgg19.preprocess_input)

Found 55347 images belonging to 39 classes.
Found 6139 images belonging to 39 classes.


In [7]:
full_model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adamax(learning_rate=0.001),
                  metrics=['acc'])
history = full_model.fit(
    train_dataset, 
    validation_data = val_dataset,
#    steps_per_epoch = 100, sabeb mau make apa ga wkwk tapi kalo pake lama
    epochs=5)

Epoch 1/5
1845/1845 [==============================] - 829s 445ms/step - loss: 1.4626 - acc: 0.6442 - val_loss: 0.3215 - val_acc: 0.9045
Epoch 2/5
1845/1845 [==============================] - 360s 195ms/step - loss: 0.2119 - acc: 0.9358 - val_loss: 0.2710 - val_acc: 0.9218
Epoch 3/5
1845/1845 [==============================] - 359s 194ms/step - loss: 0.1215 - acc: 0.9611 - val_loss: 0.2264 - val_acc: 0.9381
Epoch 4/5
1845/1845 [==============================] - 358s 194ms/step - loss: 0.0800 - acc: 0.9746 - val_loss: 0.2075 - val_acc: 0.9459
Epoch 5/5
1845/1845 [==============================] - 361s 196ms/step - loss: 0.0595 - acc: 0.9815 - val_loss: 0.2008 - val_acc: 0.9482


In [8]:
# Save weights to a file
full_model.save_weights('weights\\plant_disease_weights')
full_model.save('weights\\plant_disease')

INFO:tensorflow:Assets written to: weights\plant_disease\assets
